In [ ]:
#for BigBrain, use niwrap3912 for the conda environment
#need to import a function from a file name ../utils.py
#before running this script, run the calculate_surfnorm_coords.ipynb to generate the surfnorm and coord files 
#at the specifc surface you are interest in
#once this is run, use generate_layer_intensity_profile function to generate the intensity profile at the specifc surface 
#092325 DJ
import sys
import os
import nibabel as nib
import numpy as np
# Add parent of current working directory
parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)
from utils import generate_layer_intensity_profile

#these vol_path and surface_path are from the BigBrain dataset
vol_path = '/Users/dennis.jungchildmind.org/Desktop/BigBrain/HistologicalSpaceData/full16_100um_optbal.nii'#use the highest resolution volume 
surface_path = '/Users/dennis.jungchildmind.org/Desktop/BigBrain/PlosBiology2020gii/'


#params for the mainscript
params = {'sort_by_ap': False, 'spacing_mm': 0.24, 'dist_max_mm': 0.96, 'clim_max': 3, 'do_diff':True, 'dist_method':0, 'fontsize': 8}
HEMI = 'rh'
LAYER_TYPE = 'inf' # parameters can be 'inf', 'pial', 'white'
LAYER_SURFACE = []
SAVE_SUBPATH = []
if LAYER_TYPE == 'inf':
    LAYER_SURFACE = 'layer3'#layer 3 is set because the ex vivo is at the layer 3 as well
    SAVE_SUBPATH = 'at_inf_surface'
elif LAYER_TYPE == 'pial':
    LAYER_SURFACE = 'layer0'
    SAVE_SUBPATH = 'at_pial_surface'
elif LAYER_TYPE == 'white':
    LAYER_SURFACE = 'layer6'
    SAVE_SUBPATH = 'at_white_surface'

#set the surface path for the specifc layer type
inf_surface_lh = f'surfgii/{LAYER_SURFACE}_left_327680.surf.gii'# layer3 boundary is equivalent to the inf surface of ex vivo data    
inf_surface_rh = f'surfgii/{LAYER_SURFACE}_right_327680.surf.gii'
#make these surfnorm and coord files using the calculate_surface_normal_and_coordinates.py script
#or you can just do by workbench command
inf_surfnorm_lh = f'surfgii/{LAYER_SURFACE}_left_327680.surfnorm.func.gii'
inf_surfnorm_rh = f'surfgii/{LAYER_SURFACE}_right_327680.surfnorm.func.gii'
inf_coord_lh = f'surfgii/{LAYER_SURFACE}_left_327680.coord.func.gii'
inf_coord_rh = f'surfgii/{LAYER_SURFACE}_right_327680.coord.func.gii'
save_path_subject = '/Users/dennis.jungchildmind.org/Desktop/BigBrain/PlosBiology2020gii/'


vol = nib.load(vol_path)
# Check if required surface files exist


if HEMI == 'lh':
    surf_norm_path = os.path.join(surface_path, inf_surfnorm_lh)
    surf_coord_path = os.path.join(surface_path, inf_coord_lh)
elif HEMI == 'rh':
    surf_norm_path = os.path.join(surface_path, inf_surfnorm_rh)
    surf_coord_path = os.path.join(surface_path, inf_coord_rh)
else:
    print(f"Warning: {HEMI} not found, skipping {LAYER_SURFACE}")
    exit()

if not os.path.exists(surf_norm_path):
    print(f"Warning: {surf_norm_path} not found, skipping {LAYER_SURFACE}")

if not os.path.exists(surf_coord_path):
    print(f"Warning: {surf_coord_path} not found, skipping {LAYER_SURFACE}")

    
print(f"Processing layer: {LAYER_SURFACE}")
#calculate intensity profile
all_values, dist_array, ap_order = generate_layer_intensity_profile(
    vol, LAYER_SURFACE,
    surf_norm_path,
    surf_coord_path,
    save_path_subject, **params
)
#create the folder if not exist
save_folder = f'{save_path_subject}/{SAVE_SUBPATH}'
os.makedirs(save_folder, exist_ok=True)

#save all_values, dist_array, ap_order as npz 
np.savez(f'{save_folder}/bigbrain_{HEMI}_{LAYER_SURFACE}_{int(params["spacing_mm"]*1000)}um_max_{int(params["dist_max_mm"]*1000)}um_method{int(params["dist_method"])}_manual_raw_intensity.npz', all_values=all_values, dist_array=dist_array, ap_order=ap_order)